# Summarize results at cell-type level for the purpose of contrasting results across species
1. Proportion of FDR < 10% at various parameter for scDRS.
2. Cell type level p-value for scDRS
3. Geary's C statistics for 10kb, 1000 genes default settings.
4. LDSC-SEG p-value 

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os, sys
import scTRS.data_loader as dl
import pandas as pd
import numpy as np
from os.path import join
from statsmodels.stats.multitest import multipletests
import yaml
import scanpy as sc

sys.path.append("/n/home12/khou/holystore/")
import paper_utils

import glob
from scipy import stats
from scipy.stats import rankdata

In [2]:
DATA_PATH = "/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data"

dict_dset = {
    "tms_facs": join(
        DATA_PATH, "tabula_muris_senis/tabula-muris-senis-facs-official-raw-obj.h5ad"
    ),
    "tms_droplet": join(
        DATA_PATH, "tabula_muris_senis/tabula-muris-senis-droplet-official-raw-obj.h5ad"
    ),
    "ts_facs": join(
        DATA_PATH, "single_cell_data/tabula_sapiens/obj_smartseq2_raw.h5ad"
    ),
}

dict_df_obs = {k: sc.read_h5ad(dict_dset[k]).obs for k in dict_dset}

URL_SUPP_TABLE = "supp_tables.xlsx"

df_trait_info = pd.read_excel(
    URL_SUPP_TABLE,
    sheet_name=0,
)

df_celltype_info = pd.read_excel(
    URL_SUPP_TABLE,
    sheet_name=1,
)

In [10]:
def calc_mc_pval(dset, list_trait, stats="percentile_95"):
    SCORE_PATH = f"/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/score_file/score.{dset}_with_cov.magma_10kb_1000"

    dict_dset = {
        "tms_facs": join(
            DATA_PATH,
            "tabula_muris_senis/tabula-muris-senis-facs-official-raw-obj.h5ad",
        ),
        "tms_droplet": join(
            DATA_PATH,
            "tabula_muris_senis/tabula-muris-senis-droplet-official-raw-obj.h5ad",
        ),
        "ts_facs": join(
            DATA_PATH, "single_cell_data/tabula_sapiens/obj_smartseq2_raw.h5ad"
        ),
    }

    df_obs = sc.read_h5ad(dict_dset[dset]).obs
    df_obs.cell_ontology_class = [
        c.replace(" ", "_").replace(",", "") for c in df_obs.cell_ontology_class
    ]

    assert stats in ["max", "percentile_99", "percentile_95", "percentile_90"]
    col_ctrl_norm_score = [f"ctrl_norm_score_{i}" for i in range(1000)]

    dict_mc_pval = {}
    for trait in list_trait:
        df_tmp = pd.read_csv(
            join(SCORE_PATH, f"{trait}.full_score.gz"), sep="\t", index_col=0
        )
        df_tmp = pd.merge(
            df_obs[["cell_ontology_class"]], df_tmp, left_index=True, right_index=True
        )

        if stats == "max":
            stats_func = lambda x: np.max(x)
        elif stats == "percentile_99":
            stats_func = lambda x: np.percentile(x, 99)
        elif stats == "percentile_95":
            stats_func = lambda x: np.percentile(x, 95)
        elif stats == "percentile_90":
            stats_func = lambda x: np.percentile(x, 90)
        else:
            raise NotImplementedError
        df_tmp = df_tmp.groupby("cell_ontology_class").agg(
            {
                trait: lambda x: stats_func(x)
                for trait in ["norm_score"] + col_ctrl_norm_score
            }
        )

        rank_score = rankdata(df_tmp.values, axis=1)
        mc_pval = pd.Series(
            1 - (rank_score[:, 0] - 1) / rank_score.shape[1], index=df_tmp.index
        )
        dict_mc_pval[trait] = mc_pval
    df_mc_pval = pd.DataFrame(dict_mc_pval).T
    df_mc_pval.columns.name = None
    return df_mc_pval


import submitit

executor = submitit.AutoExecutor(folder="~/submitit/")
executor.update_parameters(timeout_min=45, mem_gb=12, slurm_partition="shared")

dict_jobs = {}
for dset in ["tms_facs", "tms_droplet", "ts_facs"]:
    jobs = executor.map_array(
        lambda x: calc_mc_pval(dset, x, stats="percentile_95"),
        np.array_split(df_trait_info["Trait_Identifier"].values, 30),
    )
    df_pval = pd.concat(j.result() for j in jobs)
    df_pval.to_csv(f"data/summary_ct/df_pval.{dset}.csv")

# Summarize Geary's C

In [3]:
trait_list = df_trait_info.Trait_Identifier.values

In [7]:
def summarize_gearysc(trait_list, tissue_list, rls_dir):
    dict_df_gearysc = dict()
    for trait in trait_list:
        df_trait = []
        for tissue in tissue_list:
            df_tmp = pd.read_csv(join(rls_dir, f"{trait}.{tissue}.csv"), index_col=0)
            df_tmp.index = [c.replace(" ", "_").replace(",", "") for c in df_tmp.index]
            df_tmp.index = f"{tissue}" + "." + df_tmp.index
            df_trait.append(df_tmp)
        df_trait = pd.concat(df_trait)
        dict_df_gearysc[trait] = df_trait

    # use p-values to assign significance
    df_tmp = pd.concat(
        [dict_df_gearysc[trait]["pval"] for trait in dict_df_gearysc], axis=1
    ).dropna()
    df_tmp.columns = [trait for trait in dict_df_gearysc]
    df_tmp["tissue"] = [i.split(".")[0] for i in df_tmp.index]
    df_tmp["ct"] = [i.split(".")[1] for i in df_tmp.index]

    # Geary's C signfinificance, first calculated within cell-type
    # Then aggregated across cell-types across tissues
    # We assign an FDR for each cell-type trait pair
    df_gearysc_meta_fdr = {}
    # calculate p-value
    for ct, df_ct in df_tmp.groupby("ct"):
        df_gearysc_meta_fdr[ct] = [
            stats.combine_pvalues(df_ct[col])[1] for col in trait_list
        ]

    df_gearysc_meta_fdr = pd.DataFrame(df_gearysc_meta_fdr, index=trait_list).T

    # convert to FDR
    df_gearysc_meta_fdr = pd.DataFrame(
        multipletests(df_gearysc_meta_fdr.values.flatten(), method="fdr_bh")[1].reshape(
            df_gearysc_meta_fdr.shape
        ),
        index=df_gearysc_meta_fdr.index,
        columns=df_gearysc_meta_fdr.columns,
    ).T
    return df_gearysc_meta_fdr


for dset in ["tms_facs", "ts_facs", "tms_droplet"]:
    df_gearysc = summarize_gearysc(
        trait_list,
        dict_df_obs[dset].tissue.unique(),
        f"02_calc_gearysc/gearysc/{dset}/",
    )
    df_gearysc.to_csv(f"data/summary_ct/df_gearysc_fdr.{dset}.csv")

# Summarize FDR proportion

In [18]:
for dset in ["tms_facs", "ts_facs", "tms_droplet"]:
    SCORE_PATH = f"/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/score_file/score.{dset}_with_cov.magma_10kb_1000"
    dict_df = dict()
    for trait in df_trait_info["Trait_Identifier"].values:
        df_temp = pd.read_csv(
            join(SCORE_PATH, f"{trait}.score.gz"), sep="\t", index_col=0
        )
        dict_df[trait] = df_temp["pval"]
    df_trs_pval = pd.DataFrame(dict_df)
    df_obs = dict_df_obs[dset]
    df_obs.cell_ontology_class = [
        c.replace(" ", "_").replace(",", "") for c in df_obs.cell_ontology_class
    ]
    df_fdr_prop = paper_utils.agg_trs_pval(
        df_obs=df_obs,
        df_pval=df_trs_pval,
        stats="fdr_prop",
        groupby="cell_ontology_class",
        fdr_prop_threshold=0.1,
    )
    # adjust order
    df_fdr_prop.to_csv(f"data/summary_ct/drs_fdr_prop.{dset}.csv")

# scDRS Proportion of FDR < 0.1 for various parameter settings

In [ ]:
for param in [
    "10kb.100",
    "10kb.500",
    "10kb.1000",
    "10kb.2000",
    "0kb.1000",
    "50kb.1000",
]:
    SCORE_PATH = f"00_calc_score/tms_facs/score_file/{param}/"
    dict_df = dict()
    for trait in df_trait_info["Trait_Identifier"].values:
        df_temp = pd.read_csv(
            join(SCORE_PATH, f"{trait}.score.gz"), sep="\t", index_col=0
        )
        dict_df[trait] = df_temp["pval"]
    df_trs_pval = pd.DataFrame(dict_df)

    df_fdr_prop = paper_utils.agg_trs_pval(
        df_obs=data_facs_ct.obs.copy(),
        df_pval=df_trs_pval,
        stats="fdr_prop",
        groupby="cell_ontology_class",
        fdr_prop_threshold=0.1,
    )
    # adjust order
    df_fdr_prop = df_fdr_prop.loc[:, df_celltype_info.id]
    df_fdr_prop.to_csv(f"./summary_ct/drs_fdr_prop.{param}.csv")

# LDSC-SEG

In [ ]:
ldsc_result_dir = "01_calc_ldsc/out/cts_result/"
df_ldsc_pval = pd.DataFrame(index=df_fdr_prop.index, columns=df_fdr_prop.columns)
for trait in df_fdr_prop.index:
    rls = pd.read_csv(
        join(ldsc_result_dir, f"{trait}.cell_type_results.txt"),
        sep="\t",
    )
    df_ldsc_pval.loc[trait, :] = rls.set_index("Name").loc[
        df_ldsc_pval.columns, "Coefficient_P_value"
    ]

df_ldsc_pval = df_ldsc_pval.astype(float)
df_ldsc_pval.to_csv("summary_ct/ldsc_seg_pval.csv")